Intent estimation import

In [1]:
import sys
from srcmodules.Scenes import Scene, SceneCoppeliaInterface
from srcmodules.Actions import Actions
from srcmodules.RobotActions import RobotActions, act
from copy import deepcopy
import numpy as np
import os

Teleop gesture toolbox import (gestures and simulator)

In [2]:
sys.path.append(os.path.abspath("./../../coppelia_sim_ros_interface/coppelia_sim_ros_interface"))
from coppelia_sim_ros_client import CoppeliaROSInterface, CoppeliaROSInterfaceNode

In [3]:
import rclpy
rclpy.init()
cop = CoppeliaROSInterfaceNode().r
from geometry_msgs.msg import Point, Pose, Quaternion

[CoppeliaROSInterface] Creating new ROS node
[Coppelia Client] Waiting for Coppelia to be initialized
[Coppelia Client] Coppelia initialized!


In [4]:
sci = SceneCoppeliaInterface(cop)

In [10]:
RobotActions.reset__not_general(sci, cop)

Object removed on the scene? Press enter


True

# Tests:

### Open drawer:

In [6]:
s = Scene(init='drawer')
s.drawer.opened = False
sci.new_scene(s)

In [7]:
sci.new_observation(s,'drawer')

In [8]:
Actions.do(s, ('open', 'drawer'), ignore_location=True)

True

In [9]:
RobotActions.open(s, cop, 'drawer')

1/5, enter to continue, anystr else to abort!
2/5, enter to continue, anystr else to abort!
3/5, enter to continue, anystr else to abort!
4/5, enter to continue, anystr else to abort!
5/5, enter to continue, anystr else to abort!


In [10]:
cop.go_to_pose(s.position_real([2,3,1]))

### Close drawer

In [8]:
s = Scene(init='drawer')
s.drawer.opened = True
sci.new_scene(s)

In [9]:
sci.new_observation(s,'drawer')

In [10]:
Actions.do(s, ('close', 'drawer'), ignore_location=True)

True

In [11]:
RobotActions.close(s, cop, 'drawer')

1/5, enter to continue, anystr else to abort!
2/5, enter to continue, anystr else to abort!
3/5, enter to continue, anystr else to abort!
4/5, enter to continue, anystr else to abort!
5/5, enter to continue, anystr else to abort!


True

### Pour

In [11]:
s = Scene(init='cup,cup')
sci.new_scene(s)

Object removed on the scene? Press enter


In [12]:
sci.new_observation(s,'cup')

In [13]:
Actions.do(s, ('pick_up', 'cup'), ignore_location=True)

True

In [14]:
RobotActions.pick_up(s,cop,'cup')

1/4, enter to continue, anystr else to abort!
2/4, enter to continue, anystr else to abort!
3/4, enter to continue, anystr else to abort!
4/4, enter to continue, anystr else to abort!


True

In [16]:
sci.new_observation(s,'cup1')

In [17]:
Actions.do(s, ('pour', 'cup1'), ignore_location=True)

False

In [18]:
RobotActions.pour(s,cop,'cup1')

1/3, enter to continue, anystr else to abort!
2/3, enter to continue, anystr else to abort!
3/3, enter to continue, anystr else to abort!


True

### Move tests

In [ ]:
s = Scene(init='drawer,object,cup')
sci.new_scene(s)

Object removed on the scene? Press enter
Scene finished -> press Enter


In [ ]:
act(s,cop,('move_down','cup'))

In [13]:
RobotActions.move_front(s,cop,'')

1/1, enter to continue, anystr else to abort!


True

In [14]:
sci.new_observation([0.4,0.0,0.4])

In [6]:
s = Scene(init='cup,object')
sci.new_scene(s)

[WARN] [1662907645.916926]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


Possibility to list all possible actions:

In [60]:
Actions.get_possible_actions(s, ignore_location=True)

[('put', 'object')]

Few of the examples:

In [ ]:
if not RobotActions.pick_object(s, 'cup'): raise Exception("Not successfull")
if not Actions.do(s, ('pick_up', 'cup'), ignore_location=True): raise Exception("Not successfull")

if not RobotActions.put(s, 'object'): raise Exception("Not successfull")
if not Actions.do(s, ('put', 'object'), ignore_location=True): raise Exception("Not successfull")
    
if not RobotActions.pick_object(s, 'object'): raise Exception("Not successfull")
if not Actions.do(s, ('pick_up', 'cup'), ignore_location=True): raise Exception("Not successfull")

### Pick up

In [7]:
# Create scene with opened drawer
s = Scene(init='cup', random=False)
s.cup.position = np.array([2,1,0])
s.cup.size = 0.05
s.r.eef_position = np.array([2,1,2])

sci.new_scene(s)

In [8]:
act(s, cop, ('pick_up', 'cup'))

1/4, enter to continue, anystr else to abort!
2/4, enter to continue, anystr else to abort!
3/4, enter to continue, anystr else to abort!
4/4, enter to continue, anystr else to abort!


True

In [9]:
act(s, cop, ('move_up', 'cup'))

1/1, enter to continue, anystr else to abort!


True

In [12]:
act(s, cop, ('move_down', 'cup'))

1/1, enter to continue, anystr else to abort!


True

### Placing

In [17]:
RobotActions.reset__not_general(sci, cop)

Object removed on the scene? Press enter


True

# Subscriber!

In [1]:
import sys
from srcmodules.Scenes import Scene, SceneCoppeliaInterface
from srcmodules.Actions import Actions
from srcmodules.RobotActions import RobotActions, act
from copy import deepcopy
import numpy as np
import os


from context_based_gesture_operation.msg import Act

Teleop gesture toolbox import (gestures and simulator)

In [2]:
sys.path.append(os.path.abspath("./../coppelia_sim_ros_interface/src"))
from coppelia_sim_ros_client import CoppeliaROSInterface

In [3]:
import rospy
rospy.init_node("main", anonymous=True)

In [4]:
cop = CoppeliaROSInterface()
from geometry_msgs.msg import Point, Pose, Quaternion

In [5]:
sci = SceneCoppeliaInterface(cop)

In [6]:
RobotActions.reset__not_general(sci, cop)

True

In [7]:
RobotActions.execution_feedback = 'time'

In [9]:
s = Scene(init='object')

In [10]:
def act__ROS_callback(data):
    target_action = data.target_action
    target_object = data.target_object
    auxiliary_parameters = data.auxiliary_parameters
    
    act(s, cop, (target_action, target_object))

rospy.Subscriber("/act", Act, act__ROS_callback)
    

[ERROR] [1664895752.859985]: bad callback: <function act__ROS_callback at 0x7ff8600bbd30>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python3.8/site-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_24544/844978657.py", line 6, in act__ROS_callback
    act(s, cop, (target_action, target_object))
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/RobotActions.py", line 15, in act
    if not Actions.do(s, TaTo, ignore_location=True): raise Exception("Action not valid!")
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/Actions.py", line 109, in do
    ret = getattr(Actions, action[0])(s, o, p, ignore_location=ignore_location)
AttributeError: type object 'Actions' has no attribute ''

[ERROR] [1664895752.864776]: bad callback: <function act__ROS_callback at 0x7ff838723940>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python

[ERROR] [1664895755.819003]: bad callback: <function act__ROS_callback at 0x7ff8600bbd30>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python3.8/site-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_24544/844978657.py", line 6, in act__ROS_callback
    act(s, cop, (target_action, target_object))
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/RobotActions.py", line 15, in act
    if not Actions.do(s, TaTo, ignore_location=True): raise Exception("Action not valid!")
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/Actions.py", line 109, in do
    ret = getattr(Actions, action[0])(s, o, p, ignore_location=ignore_location)
AttributeError: type object 'Actions' has no attribute ''

[ERROR] [1664895755.824122]: bad callback: <function act__ROS_callback at 0x7ff838723940>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python

[ERROR] [1664895758.830796]: bad callback: <function act__ROS_callback at 0x7ff8600bbd30>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python3.8/site-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_24544/844978657.py", line 6, in act__ROS_callback
    act(s, cop, (target_action, target_object))
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/RobotActions.py", line 15, in act
    if not Actions.do(s, TaTo, ignore_location=True): raise Exception("Action not valid!")
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/Actions.py", line 109, in do
    ret = getattr(Actions, action[0])(s, o, p, ignore_location=ignore_location)
AttributeError: type object 'Actions' has no attribute ''

[ERROR] [1664895758.834867]: bad callback: <function act__ROS_callback at 0x7ff838723940>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python

[ERROR] [1664895761.842498]: bad callback: <function act__ROS_callback at 0x7ff8600bbd30>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python3.8/site-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_24544/844978657.py", line 6, in act__ROS_callback
    act(s, cop, (target_action, target_object))
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/RobotActions.py", line 15, in act
    if not Actions.do(s, TaTo, ignore_location=True): raise Exception("Action not valid!")
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/Actions.py", line 109, in do
    ret = getattr(Actions, action[0])(s, o, p, ignore_location=ignore_location)
AttributeError: type object 'Actions' has no attribute ''

[ERROR] [1664895761.847035]: bad callback: <function act__ROS_callback at 0x7ff838723940>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python

[ERROR] [1664895764.854992]: bad callback: <function act__ROS_callback at 0x7ff8600bbd30>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python3.8/site-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_24544/844978657.py", line 6, in act__ROS_callback
    act(s, cop, (target_action, target_object))
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/RobotActions.py", line 15, in act
    if not Actions.do(s, TaTo, ignore_location=True): raise Exception("Action not valid!")
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/Actions.py", line 109, in do
    ret = getattr(Actions, action[0])(s, o, p, ignore_location=ignore_location)
AttributeError: type object 'Actions' has no attribute ''

[ERROR] [1664895764.859359]: bad callback: <function act__ROS_callback at 0x7ff838723940>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python

[ERROR] [1664895767.866984]: bad callback: <function act__ROS_callback at 0x7ff8600bbd30>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python3.8/site-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "/tmp/ipykernel_24544/844978657.py", line 6, in act__ROS_callback
    act(s, cop, (target_action, target_object))
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/RobotActions.py", line 15, in act
    if not Actions.do(s, TaTo, ignore_location=True): raise Exception("Action not valid!")
  File "/home/petr/my_ws/src/context_based_gesture_operation/srcmodules/Actions.py", line 109, in do
    ret = getattr(Actions, action[0])(s, o, p, ignore_location=ignore_location)
AttributeError: type object 'Actions' has no attribute ''

[ERROR] [1664895767.872116]: bad callback: <function act__ROS_callback at 0x7ff838723940>
Traceback (most recent call last):
  File "/home/petr/anaconda3/envs/robostackenv/lib/python

In [12]:
cop